In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500
pd.options.display.max_rows=500

In [2]:
import os

In [3]:
from pycountry import countries

In [4]:
pwd

'/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/jupyter_notebooks'

In [5]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [6]:
from hopkins_cleaner import HopkinsDataFull
from us_data_cleaner import USDataCleanUp

In [15]:
self = HopkinsDataFull()

In [16]:
self.run()

In [17]:
df = self.data

In [18]:
df.date.max()

Timestamp('2020-03-19 00:00:00')

In [19]:
df.loc[df['province_or_state'].str.contains('DC')].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
15581,US,2020-03-15,DC,16,DC-US,10.0,6.0,0,0.0,0.0,...,-77.0369,6.0,42.0,US,USA,327167434.0,38.1,3499,NaN,9.0
15582,US,2020-03-16,DC,22,DC-US,16.0,6.0,0,0.0,0.0,...,-77.0369,7.0,43.0,US,USA,327167434.0,38.1,4632,NaN,10.0
15583,US,2020-03-17,DC,22,DC-US,22.0,0.0,0,0.0,0.0,...,-77.0369,8.0,44.0,US,USA,327167434.0,38.1,6421,NaN,11.0
15584,US,2020-03-18,DC,31,DC-US,22.0,9.0,0,0.0,0.0,...,-77.0369,9.0,45.0,US,USA,327167434.0,38.1,7783,NaN,12.0
15585,US,2020-03-19,DC,40,DC-US,31.0,9.0,0,0.0,0.0,...,-77.0369,10.0,46.0,US,USA,327167434.0,38.1,13677,NaN,13.0


In [ ]:
df.to_csv('../output_data/HOPKINS_CLEANED.csv', index=False)

In [ ]:
df

# QA 

#### Make sure all US states have the same number of days 

In [20]:
us = df.loc[df['country_or_region'] == 'US']

In [21]:
us.groupby(['province_or_state'])['daily_new_cases'].size().reset_index()

,province_or_state,daily_new_cases
0,AK,58
1,AL,58
2,AR,58
3,AZ,58
4,CA,58
5,CO,58
6,CT,58
7,DC,58
8,DE,58
9,Diamond Princess,58


#### Total US cases 

In [22]:
us['daily_new_cases'].sum()

13677.0

In [23]:
us.groupby(['date'])['daily_new_cases'].sum().reset_index()

,date,daily_new_cases
0,2020-01-22,1.0
1,2020-01-23,0.0
2,2020-01-24,1.0
3,2020-01-25,0.0
4,2020-01-26,3.0
5,2020-01-27,0.0
6,2020-01-28,0.0
7,2020-01-29,0.0
8,2020-01-30,0.0
9,2020-01-31,2.0


In [24]:
df.head(2)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
0,Thailand,2020-01-22,Not Provided,2,Not Provided-Thailand,0.0,2.0,0,0.0,0.0,...,101.0,NaN,NaN,TH,THA,69428524.0,37.7,2,NaN,NaN
1,Thailand,2020-01-23,Not Provided,3,Not Provided-Thailand,2.0,1.0,0,0.0,0.0,...,101.0,NaN,NaN,TH,THA,69428524.0,37.7,3,NaN,NaN


In [25]:
max_date = df['date'].max()

In [26]:
yesterday = df.loc[df['date'] == max_date]
yesterday['running_total_cases'].sum(), yesterday['running_total_recoveries'].sum(), yesterday['running_total_deaths'].sum()

(242708, 84854, 9867)

In [27]:
df['daily_new_cases'].sum(), df['daily_new_recoveries'].sum(), df['daily_new_deaths'].sum()

(242708.0, 84854.0, 9867.0)

In [28]:
df.loc[df['province_or_state'] == 'WA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
16031,US,2020-03-15,WA,643,WA-US,572.0,71.0,1,1.0,0.0,...,-120.740135,16.0,42.0,US,USA,327167434.0,38.1,3499,9.0,9.0
16032,US,2020-03-16,WA,904,WA-US,643.0,261.0,1,1.0,0.0,...,-120.740135,17.0,43.0,US,USA,327167434.0,38.1,4632,10.0,10.0
16033,US,2020-03-17,WA,1076,WA-US,904.0,172.0,1,1.0,0.0,...,-120.740135,18.0,44.0,US,USA,327167434.0,38.1,6421,11.0,11.0
16034,US,2020-03-18,WA,1014,WA-US,1076.0,-62.0,0,1.0,-1.0,...,-120.740135,19.0,45.0,US,USA,327167434.0,38.1,7783,12.0,12.0
16035,US,2020-03-19,WA,1376,WA-US,1014.0,362.0,0,0.0,0.0,...,-120.740135,20.0,46.0,US,USA,327167434.0,38.1,13677,13.0,13.0


In [29]:
df.loc[df['province_or_state'] == 'VA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
16001,US,2020-03-15,VA,45,VA-US,41.0,4.0,0,0.0,0.0,...,-78.024902,6.0,42.0,US,USA,327167434.0,38.1,3499,NaN,9.0
16002,US,2020-03-16,VA,49,VA-US,45.0,4.0,0,0.0,0.0,...,-78.024902,7.0,43.0,US,USA,327167434.0,38.1,4632,NaN,10.0
16003,US,2020-03-17,VA,67,VA-US,49.0,18.0,0,0.0,0.0,...,-78.024902,8.0,44.0,US,USA,327167434.0,38.1,6421,NaN,11.0
16004,US,2020-03-18,VA,77,VA-US,67.0,10.0,0,0.0,0.0,...,-78.024902,9.0,45.0,US,USA,327167434.0,38.1,7783,NaN,12.0
16005,US,2020-03-19,VA,99,VA-US,77.0,22.0,0,0.0,0.0,...,-78.024902,10.0,46.0,US,USA,327167434.0,38.1,13677,NaN,13.0


In [ ]:
df.loc[df['province_or_state'] == 'CA'].tail(5)

In [ ]:
yesterday.sort_values('running_total_cases', ascending=False)[['country_or_region','running_total_cases','running_total_recoveries','running_total_deaths','country_population_2018','country_median_age']]

## QA of Data 

In [ ]:
df.head(2)

In [ ]:
# This indicates a shift in the data
df.loc[df['daily_new_cases'] < 0]['date'].value_counts()

In [ ]:
# This indicates a shift in the data
us.loc[us['daily_new_cases'] < 0]

In [ ]:
us['daily_new_cases'].sum(), us['daily_new_recoveries'].sum(), us['daily_new_deaths'].sum()

In [ ]:
# This indicates a shift in the data
df.loc[df['daily_new_cases'] < 0]['province_or_state'].value_counts()

In [ ]:
df.loc[df['daily_new_cases'] < 0]

In [ ]:
df.loc[(df['daily_new_cases'] < 0) &
      (df['province_or_state'] == 'CA')]

In [ ]:
df.loc[df['country_or_region'] == 'China'].tail(5)

In [ ]:
df.country_or_region.value_counts()